In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2520)

In [15]:
x[:,:2]

matrix([[ 344351.69113349,  335268.12872603],
        [ 352726.79180001,  347914.61351239],
        [ 623863.56776834,  621320.65346466],
        [ 675760.8456823 ,  665257.54347434],
        [ 182200.39293639,  176117.30306953],
        [ 212449.92803494,  213353.22407784],
        [ 399861.72209966,  395233.14633024],
        [ 415973.43897655,  405855.28773562],
        [ 404787.41252918,  388750.46079851],
        [ 399919.47030913,  410246.60194474],
        [ 406834.23826693,  414651.98651323],
        [ 349663.27045432,  344651.80951894],
        [  93659.85358612,   94857.02506913],
        [ 100186.55057092,  103762.44884542],
        [ 158429.54710944,  148697.76176325],
        [ 147172.13955102,  150008.27631656],
        [ 206017.97799556,  204909.98828895],
        [ 201718.40945513,  206795.32482742],
        [ 376783.44925823,  361533.00315103],
        [ 360170.44610244,  365184.84854169],
        [ 540114.1244676 ,  551706.39944653],
        [ 566306.87396364,  567899

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-02, 4e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.07796481e+07  0.00000000e+00  5.01e+03 1.64e+03  1.00e+06     0s
   1  -2.56567129e+07 -1.21774364e+07  1.15e+03 3.76e+02  2.75e+05     0s
   2  -1.87608427e+07 -2.12841826e+07  3.23e+01 1.06e+01  6.11e+04     0s
   3  -1.91064609e+07 -1.93323474e+07  1.63e-02 5.33e-03  4.0

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.9048031061419927e-10,
 322800.69642312406,
 65823.14699234409,
 1.199721774042599e-07,
 149455.22787254216,
 2.0146816928882526e-07,
 2.0231231923249028e-07,
 0.00010022237454825403,
 1.1794432141344892e-11,
 272257.0418140794,
 1.1793272747892243e-11,
 1.179443214138377e-11,
 1.1793741707267005e-11,
 1.1873245323620933e-11,
 457478.855511395,
 1082.2360582291933,
 1.199721334164871e-07,
 1.19972176037989e-07,
 249870.5158260762,
 428320.73975656886,
 2.0141223210907815e-07,
 207761.9922143385,
 162758.03200494678,
 70835.55993450715,
 2.8317002711729204e-05,
 2.831877259660675e-05,
 2.86483958425671e-05,
 188756.19066296075,
 0.03509605868262167,
 0.012910401645960246,
 125358.34094475288,
 19491.350428242615,
 22544.81528282214,
 5.388646402368055e-09,
 12313.418382575297,
 192337.99780019993,
 989.8117268725439,
 1.4596732814818234e-06,
 24801.57808119422,
 195661.0358104946,
 425141.66038653796,
 26133.86533602454,
 4.345482111746358e-06,
 4.345271647097069e-06,
 1.7839582220738